# Create a triangulation from an ordered sequence of vertices



In [46]:
infile = 'austria.csv' # filename for the file containing the vertices
inner_track_name = 'inner_track' # name for the inner track
inner_track_filename = inner_track_name + '.poly' # filename which contains input for triangle of the inner part of the track
triangle_folder = '/home/stefan/tu/repositories/simplex-architectures/cmake-build-release/triangle-prefix/src/triangle/' # triangle-executable folder
hypro_outfile = '/home/stefan/tu/repositories/simplex-architectures/src/tool_car/bad_states.h' # the file will contain definitions of polytopes for the inner part of the track

Read in the csv-file containing the vertices of the track.

In [47]:
nodes = []

with open(infile, 'r') as f:
    nodes = [ (line.split(",")[0].strip(), line.split(",")[1].strip()) for line in f.readlines() if '#' not in line]

print(str(nodes))


[('567.8', '140.464'), ('407.28', '99.8436'), ('300.37', '73.3236'), ('287.54', '72.6036'), ('278.95', '88.3236'), ('250.27', '127.734'), ('227.41', '159.864'), ('150.03', '314.124'), ('66.76', '409.744'), ('71.88', '425.194'), ('143.78', '426.974'), ('193.99', '417.434'), ('268.61', '403.814'), ('336.96', '396.914'), ('407.96', '391.444'), ('418.09', '369.944'), ('350.24', '322.614'), ('288.07', '328.394'), ('235.61', '332.144'), ('213.37', '296.674'), ('217.61', '282.944'), ('251.7', '219.664'), ('299.17', '214.164'), ('332.46', '248.944'), ('370.08', '262.144'), ('536.13', '261.774'), ('576.04', '261.754'), ('597.75', '247.034'), ('616.07', '177.874'), ('607.42', '159.384')]


Prepare a *.poly file for triangle which contains all the nodes and segments

In [48]:

out = open(inner_track_filename,'w')
# write first line: number of nodes, dimension, number of attributes, markers (1 or 0)
out.write(str(len(nodes)) + " 2 0 0\n")
# write all nodes
for i in range(len(nodes)):
    out.write(str(i) + " " + nodes[i][0] + " " + nodes[i][1] + "\n")

# now add segments, first a line indicating how many segments there will be, whether we want to have boundary markers, 
# then the segments (the two node indices defining a segment)
out.write(str(len(nodes)) + " 0\n")
for i in range(len(nodes)):
    out.write(str(i) + " " + str(i) + " " + str((i+1) % len(nodes)) + "\n")

# finally, add one line for indicating that there are no holes (with no boundary markers)
out.write("0 0")

out.close()


We can now call triangle on the created file and read in the output 

In [49]:
import os

# call to triangle
os.system(triangle_folder + "triangle -pYj " + inner_track_filename)

# collect output files
vertex_sets = []
with open(inner_track_name + ".1.poly") as f:
    vertex_sets = [ (int(line.split()[1]), int(line.split()[2]), int(line.split()[3])) for line in f.readlines() if len(line.split()) == 4]

print(str(vertex_sets))


Opening inner_track.poly.
Constructing Delaunay triangulation by divide-and-conquer method.
Delaunay milliseconds:  0
Recovering segments in Delaunay triangulation.
Segment milliseconds:  0
Removing unwanted triangles.
Hole milliseconds:  0

Writing inner_track.1.node.
Writing inner_track.1.ele.
Writing inner_track.1.poly.

Output milliseconds:  0
Total running milliseconds:  0

Statistics:

  Input vertices: 30
  Input segments: 30
  Input holes: 0

  Mesh vertices: 30
  Mesh triangles: 28
  Mesh edges: 57
  Mesh exterior boundary edges: 30
  Mesh interior boundary edges: 0
  Mesh subsegments (constrained edges): 30

[(2, 0, 1), (0, 1, 1), (1, 2, 1), (2, 3, 1), (3, 4, 1), (4, 5, 1), (6, 5, 1), (7, 6, 1), (7, 8, 1), (8, 9, 1), (9, 10, 1), (11, 10, 1), (12, 11, 1), (13, 12, 1), (14, 13, 1), (14, 15, 1), (16, 15, 1), (16, 17, 1), (18, 17, 1), (19, 18, 1), (19, 20, 1), (21, 20, 1), (22, 21, 1), (23, 22, 1), (23, 24, 1), (25, 24, 1), (25, 26, 1), (26, 27, 1), (27, 28, 1), (28, 29, 1), (29,

Create HyPro-code that creates polytopes for each of the passed triangles.

In [50]:
hyprofile = open(hypro_outfile,'w')

hyprofile.write(
'#include <hypro/representations/GeometricObjectBase.h>\n\
#include <vector>\n\n\
#pragma once\n\n\
namespace simplexArchitectures {\n\
template<typename Automaton>\n\
typename Automaton::conditionVector createBadStates() {\n\
\tusing Number = typename Automaton::NumberType;\n\
\tauto res = typename Automaton::conditionVector();\n\n')

for v_set in vertex_sets:
    hyprofile.write(
'\t{\n'
'\tstd::vector<hypro::Point<Number>> points{\n' +
'\t\thypro::Point<Number>({'+ nodes[v_set[0]][0] + ',' + nodes[v_set[0]][1] + '}),\n' +
'\t\thypro::Point<Number>({'+ nodes[v_set[1]][0] + ',' + nodes[v_set[1]][1] + '}),\n' +
'\t\thypro::Point<Number>({'+ nodes[v_set[2]][0] + ',' + nodes[v_set[2]][1] + '})\n' + 
'\t};\n\
\thypro::HPolytope<Number> poly{points};\n\
\tres.push_back(hypro::Condition<Number>(poly.matrix(), poly.vector()));\n\t}\n'
    )
    
hyprofile.write('}\n} // namespace\n')

hyprofile.close()